In [1]:
import sys
from pathlib import Path
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

# Import configuration
from config import config

In [2]:
listings_df = pd.read_csv(f"../data/{config.listings_file}")
print(f"Loaded {len(listings_df):,} listings")
print(f"Columns: {list(listings_df.columns)}")
print(f"Data types:\n{listings_df.dtypes}")

Loaded 1,428 listings
Columns: ['listing_id', 'surface_m2', 'description', 'room_count', 'floor', 'floor_count', 'current_price', 'city', 'city_zip', 'transaction_type', 'item_type']
Data types:
listing_id            int64
surface_m2            int64
description          object
room_count          float64
floor               float64
floor_count         float64
current_price         int64
city                 object
city_zip              int64
transaction_type     object
item_type            object
dtype: object


In [3]:
key_features = ['current_price', 'surface_m2', 'room_count', 'floor', 'floor_count']

print(f"Dataset shape: {listings_df.shape}")
print(f"Analyzing key features: {key_features}")

print(f"\MISSING DATA:")
for col in key_features:
    if col in listings_df.columns:
        null_count = listings_df[col].isnull().sum()
        null_pct = (null_count / len(listings_df)) * 100
        print(f"   {col:<15}: {null_count:>4} missing ({null_pct:>5.1f}%)")
    else:
        print(f"   {col:<15}: Column not found")

print(f"\nCURRENT_PRICE ANALYSIS:")
price_null = listings_df['current_price'].isnull().sum()
price_zero = (listings_df['current_price'] == 0).sum()
price_negative = (listings_df['current_price'] < 0).sum()
price_valid = len(listings_df) - price_null - price_zero - price_negative

print(f"   Null values: {price_null}")
print(f"   Zero values: {price_zero} (treating as missing)")
print(f"   Negative values: {price_negative}")
print(f"   Valid prices: {price_valid}")
print(f"   Total missing (null + zero): {price_null + price_zero} ({((price_null + price_zero) / len(listings_df)) * 100:.1f}%)")

if price_valid > 0:
    valid_prices = listings_df[listings_df['current_price'] > 0]['current_price']
    print(f"   Price range: €{valid_prices.min():,.0f} - €{valid_prices.max():,.0f}")
    print(f"   Price median: €{valid_prices.median():,.0f}")
    print(f"   Price mean: €{valid_prices.mean():,.0f}")

print(f"\nSURFACE_M2 ANALYSIS:")
surface_null = listings_df['surface_m2'].isnull().sum()
surface_zero = (listings_df['surface_m2'] == 0).sum() if 'surface_m2' in listings_df.columns else 0
surface_negative = (listings_df['surface_m2'] < 0).sum() if 'surface_m2' in listings_df.columns else 0

print(f"   Null values: {surface_null}")
print(f"   Zero values: {surface_zero}")
print(f"   Negative values: {surface_negative}")

if surface_null < len(listings_df):
    valid_surfaces = listings_df[listings_df['surface_m2'] > 0]['surface_m2']
    if len(valid_surfaces) > 0:
        print(f"   Surface range: {valid_surfaces.min():.0f}m² - {valid_surfaces.max():.0f}m²")
        print(f"   Surface median: {valid_surfaces.median():.0f}m²")
        print(f"   Surface mean: {valid_surfaces.mean():.1f}m²")

print(f"\nROOM_COUNT DETAILED ANALYSIS:")
room_null = listings_df['room_count'].isnull().sum()
print(f"   Null values: {room_null}")

if room_null < len(listings_df):
    valid_rooms = listings_df['room_count'].dropna()
    room_distribution = valid_rooms.value_counts().sort_index()
    print(f"   Room distribution:")
    for rooms, count in room_distribution.items():
        print(f"     {rooms} rooms: {count} listings ({(count/len(valid_rooms)*100):.1f}%)")

print(f"\nFLOOR DATA ANALYSIS:")
floor_null = listings_df['floor'].isnull().sum() if 'floor' in listings_df.columns else len(listings_df)
floor_count_null = listings_df['floor_count'].isnull().sum() if 'floor_count' in listings_df.columns else len(listings_df)

print(f"   Floor missing: {floor_null} ({(floor_null/len(listings_df)*100):.1f}%)")
print(f"   Floor_count missing: {floor_count_null} ({(floor_count_null/len(listings_df)*100):.1f}%)")

if 'floor' in listings_df.columns and 'floor_count' in listings_df.columns:
    both_present = listings_df[listings_df['floor'].notnull() & listings_df['floor_count'].notnull()]
    if len(both_present) > 0:
        inconsistent = both_present[both_present['floor'] > both_present['floor_count']]
        print(f"   Both floor & floor_count available: {len(both_present)}")
        print(f"   Inconsistent (floor > floor_count): {len(inconsistent)}")
        
        if len(inconsistent) > 0:
            print(f"   Examples of inconsistencies:")
            for idx in inconsistent.index[:5]:
                floor_val = both_present.loc[idx, 'floor']
                floor_count_val = both_present.loc[idx, 'floor_count']
                print(f"     Listing {both_present.loc[idx, 'listing_id']}: floor={floor_val}, floor_count={floor_count_val}")

Dataset shape: (1428, 11)
Analyzing key features: ['current_price', 'surface_m2', 'room_count', 'floor', 'floor_count']
\MISSING DATA:
   current_price  :    0 missing (  0.0%)
   surface_m2     :    0 missing (  0.0%)
   room_count     :    3 missing (  0.2%)
   floor          :  151 missing ( 10.6%)
   floor_count    :  474 missing ( 33.2%)

CURRENT_PRICE ANALYSIS:
   Null values: 0
   Zero values: 5 (treating as missing)
   Negative values: 0
   Valid prices: 1423
   Total missing (null + zero): 5 (0.4%)
   Price range: €12,000 - €485,000
   Price median: €188,000
   Price mean: €198,837

SURFACE_M2 ANALYSIS:
   Null values: 0
   Zero values: 4
   Negative values: 0
   Surface range: 11m² - 179m²
   Surface median: 63m²
   Surface mean: 62.5m²

ROOM_COUNT DETAILED ANALYSIS:
   Null values: 3
   Room distribution:
     1.0 rooms: 28 listings (2.0%)
     2.0 rooms: 263 listings (18.5%)
     3.0 rooms: 544 listings (38.2%)
     4.0 rooms: 487 listings (34.2%)
     5.0 rooms: 81 listing

In [4]:
listings_imputed = listings_df.copy()

# Mark zero prices as missing (convert to NaN)
zero_prices = (listings_imputed['current_price'] == 0).sum()
listings_imputed.loc[listings_imputed['current_price'] == 0, 'current_price'] = np.nan

print(f"Converted {zero_prices} zero prices to NaN")

# Mark zero surfaces as missing (convert to NaN)
zero_surfaces = (listings_imputed['surface_m2'] == 0).sum()
listings_imputed.loc[listings_imputed['surface_m2'] == 0, 'surface_m2'] = np.nan

print(f"Converted {zero_surfaces} zero surfaces to NaN")

# Create flags to track what was imputed
listings_imputed['price_was_imputed'] = listings_imputed['current_price'].isnull()
listings_imputed['surface_was_imputed'] = listings_imputed['surface_m2'].isnull()
listings_imputed['rooms_was_imputed'] = listings_imputed['room_count'].isnull()
listings_imputed['floor_count_was_imputed'] = listings_imputed['floor_count'].isnull() if 'floor_count' in listings_imputed.columns else True

print(f"\nMISSING VALUES TO IMPUTE:")
print(f"   current_price: {listings_imputed['current_price'].isnull().sum()}")
print(f"   surface_m2: {listings_imputed['surface_m2'].isnull().sum()}")
print(f"   room_count: {listings_imputed['room_count'].isnull().sum()}")
print(f"   floor_count: {listings_imputed['floor_count'].isnull().sum() if 'floor_count' in listings_imputed.columns else 'Column not found'}")

Converted 5 zero prices to NaN
Converted 4 zero surfaces to NaN

MISSING VALUES TO IMPUTE:
   current_price: 5
   surface_m2: 4
   room_count: 3
   floor_count: 474


In [5]:
# Find missing room counts
missing_rooms_mask = listings_imputed['room_count'].isnull()
missing_rooms_count = missing_rooms_mask.sum()

print(f"Missing room counts: {missing_rooms_count}")

if missing_rooms_count > 0:
    # Calculate mode for room_count
    room_mode = listings_imputed['room_count'].mode()
    if len(room_mode) > 0:
        mode_value = room_mode.iloc[0]
        print(f"Room_count mode: {mode_value}")
        
        # Fill all missing room counts with mode
        listings_imputed['room_count'] = listings_imputed['room_count'].fillna(mode_value)
        print(f"Filled {missing_rooms_count} missing room counts with mode: {mode_value}")
    else:
        # Fallback if no mode can be calculated
        listings_imputed['room_count'] = listings_imputed['room_count'].fillna(2)
        print(f"No mode available, used fallback value: 2")

print(f"Room count imputation complete: {listings_imputed['room_count'].isnull().sum()} remaining missing")

# Show room count distribution after imputation
room_dist_after = listings_imputed['room_count'].value_counts().sort_index()
print(f"Room distribution after imputation:")
for rooms, count in room_dist_after.items():
    print(f"   {rooms} rooms: {count} listings")

Missing room counts: 3
Room_count mode: 3.0
Filled 3 missing room counts with mode: 3.0
Room count imputation complete: 0 remaining missing
Room distribution after imputation:
   1.0 rooms: 28 listings
   2.0 rooms: 263 listings
   3.0 rooms: 547 listings
   4.0 rooms: 487 listings
   5.0 rooms: 81 listings
   6.0 rooms: 16 listings
   7.0 rooms: 6 listings


In [6]:
missing_surface_mask = listings_imputed['surface_m2'].isnull()
missing_surface_count = missing_surface_mask.sum()

print(f"Missing surface values: {missing_surface_count}")

if missing_surface_count > 0:
    imputed_surfaces = 0
    
    # Process each listing with missing surface
    for idx in listings_imputed[missing_surface_mask].index:
        room_count = listings_imputed.loc[idx, 'room_count']
        
        if pd.notna(room_count):
            # Find properties with same room count and valid surface
            similar_properties = listings_imputed[
                (listings_imputed['room_count'] == room_count) & 
                (listings_imputed['surface_m2'].notnull())
            ]['surface_m2']
            
            if len(similar_properties) >= 3:
                # Use median of similar properties
                imputed_surface = similar_properties.median()
                listings_imputed.loc[idx, 'surface_m2'] = imputed_surface
                imputed_surfaces += 1
                
                print(f"   Listing {listings_imputed.loc[idx, 'listing_id']}: {room_count} rooms → {imputed_surface:.0f}m² (from {len(similar_properties)} similar)")
    
    # For remaining missing surfaces, use overall median
    remaining_missing = listings_imputed['surface_m2'].isnull().sum()
    if remaining_missing > 0:
        overall_median = listings_imputed['surface_m2'].median()
        listings_imputed['surface_m2'] = listings_imputed['surface_m2'].fillna(overall_median)
        print(f"Filled remaining {remaining_missing} missing surfaces with overall median: {overall_median:.0f}m²")
    
    print(f"Surface imputation complete: {imputed_surfaces} room-based + {remaining_missing} median-based")
    print(f"Remaining missing surfaces: {listings_imputed['surface_m2'].isnull().sum()}")

Missing surface values: 4
   Listing 116736140: 2.0 rooms → 44m² (from 259 similar)
   Listing 64721495: 2.0 rooms → 44m² (from 260 similar)
   Listing 64728971: 2.0 rooms → 44m² (from 261 similar)
   Listing 65459581: 2.0 rooms → 44m² (from 262 similar)
Surface imputation complete: 4 room-based + 0 median-based
Remaining missing surfaces: 0


In [7]:
missing_price_mask = listings_imputed['current_price'].isnull()
missing_price_count = missing_price_mask.sum()

print(f"Missing price values: {missing_price_count}")

if missing_price_count > 0:
    imputed_prices = 0
    
    # Process each listing with missing price
    for idx in listings_imputed[missing_price_mask].index:
        surface = listings_imputed.loc[idx, 'surface_m2']
        
        if pd.notna(surface) and surface > 0:
            # Find properties with similar surfaces (±20%)
            surface_min = surface * 0.8
            surface_max = surface * 1.2
            
            similar_properties = listings_imputed[
                (listings_imputed['surface_m2'].between(surface_min, surface_max)) &
                (listings_imputed['current_price'].notnull()) &
                (listings_imputed['current_price'] > 0)
            ]['current_price']
            
            if len(similar_properties) >= 3:
                # Use median of similar properties
                imputed_price = similar_properties.median()
                listings_imputed.loc[idx, 'current_price'] = imputed_price
                imputed_prices += 1
                
                print(f"   Listing {listings_imputed.loc[idx, 'listing_id']}: {surface:.0f}m² → €{imputed_price:,.0f} (from {len(similar_properties)} similar surfaces)")
    
    print(f"Price imputation complete: {imputed_prices} properties imputed")
    remaining_missing = listings_imputed['current_price'].isnull().sum()
    print(f"Remaining missing prices: {remaining_missing} (not enough similar surfaces)")

Missing price values: 5
   Listing 35031592: 64m² → €188,000 (from 940 similar surfaces)
   Listing 46868648: 86m² → €228,000 (from 362 similar surfaces)
   Listing 68728689: 100m² → €269,900 (from 153 similar surfaces)
   Listing 70413924: 100m² → €269,900 (from 154 similar surfaces)
   Listing 97297712: 63m² → €188,000 (from 946 similar surfaces)
Price imputation complete: 5 properties imputed
Remaining missing prices: 0 (not enough similar surfaces)


In [8]:
if 'floor_count' in listings_imputed.columns:
    missing_floor_count_mask = listings_imputed['floor_count'].isnull()
    missing_floor_count = missing_floor_count_mask.sum()
    
    print(f"Missing floor_count values: {missing_floor_count}")
    
    if missing_floor_count > 0:
        # Calculate mode for floor_count
        floor_count_mode = listings_imputed['floor_count'].mode()
        if len(floor_count_mode) > 0:
            mode_value = floor_count_mode.iloc[0]
            print(f"Floor_count mode: {mode_value}")
            
            # Apply mode imputation
            listings_imputed['floor_count'] = listings_imputed['floor_count'].fillna(mode_value)
            print(f"Filled {missing_floor_count} missing floor_count values with mode: {mode_value}")
    
    # Validation: Ensure floor_count >= floor where both exist
    print(f"\nVALIDATING FLOOR_COUNT >= FLOOR:")
    
    if 'floor' in listings_imputed.columns:
        both_available = listings_imputed[
            listings_imputed['floor'].notnull() & 
            listings_imputed['floor_count'].notnull()
        ]
        
        inconsistent_mask = both_available['floor'] > both_available['floor_count']
        inconsistent_count = inconsistent_mask.sum()
        
        print(f"   Properties with both floor & floor_count: {len(both_available)}")
        print(f"   Inconsistent cases (floor > floor_count): {inconsistent_count}")
        
        if inconsistent_count > 0:
            print(f"   Fixing inconsistent cases...")
            
            for idx in both_available[inconsistent_mask].index:
                current_floor = listings_imputed.loc[idx, 'floor']
                current_floor_count = listings_imputed.loc[idx, 'floor_count']
                
                # Set floor_count to at least floor + 1 (building must have more floors than the apartment floor)
                new_floor_count = max(current_floor + 1, current_floor_count)
                listings_imputed.loc[idx, 'floor_count'] = new_floor_count
                
                print(f"     Listing {listings_imputed.loc[idx, 'listing_id']}: floor={current_floor}, floor_count {current_floor_count}→{new_floor_count}")
            
            print(f"Fixed {inconsistent_count} inconsistent floor_count values")
    
    print(f"Final missing floor_count: {listings_imputed['floor_count'].isnull().sum()}")
else:
    print("floor_count column not found in dataset")

Missing floor_count values: 474
Floor_count mode: 4.0
Filled 474 missing floor_count values with mode: 4.0

VALIDATING FLOOR_COUNT >= FLOOR:
   Properties with both floor & floor_count: 1277
   Inconsistent cases (floor > floor_count): 50
   Fixing inconsistent cases...
     Listing 122096889: floor=1.0, floor_count 0.0→2.0
     Listing 57140221: floor=1.0, floor_count 0.0→2.0
     Listing 121050026: floor=4.0, floor_count 0.0→5.0
     Listing 121214547: floor=3.0, floor_count 0.0→4.0
     Listing 122651930: floor=1.0, floor_count 0.0→2.0
     Listing 122411343: floor=1.0, floor_count 0.0→2.0
     Listing 121137121: floor=3.0, floor_count 0.0→4.0
     Listing 121050030: floor=3.0, floor_count 0.0→4.0
     Listing 119989706: floor=3.0, floor_count 0.0→4.0
     Listing 116517392: floor=2.0, floor_count 0.0→3.0
     Listing 120094259: floor=4.0, floor_count 0.0→5.0
     Listing 119989703: floor=4.0, floor_count 0.0→5.0
     Listing 115106873: floor=2.0, floor_count 0.0→3.0
     Listing 18

In [9]:
# Compare before and after
features_summary = []

for feature in ['current_price', 'surface_m2', 'room_count', 'floor_count']:
    if feature in listings_df.columns:
        # Original missing (treating 0 as missing for price)
        if feature == 'current_price':
            original_missing = listings_df[feature].isnull().sum() + (listings_df[feature] == 0).sum()
        else:
            original_missing = listings_df[feature].isnull().sum()
        
        # After imputation
        final_missing = listings_imputed[feature].isnull().sum()
        
        # Success metrics
        total_rows = len(listings_df)
        imputed_count = original_missing - final_missing
        success_rate = (imputed_count / original_missing * 100) if original_missing > 0 else 100
        
        features_summary.append({
            'feature': feature,
            'original_missing': original_missing,
            'final_missing': final_missing,
            'imputed_count': imputed_count,
            'success_rate': success_rate
        })
        
        print(f"{feature}:")
        print(f"   Before: {original_missing:>4} missing ({original_missing/total_rows*100:>5.1f}%)")
        print(f"   After:  {final_missing:>4} missing ({final_missing/total_rows*100:>5.1f}%)")
        print(f"   Imputed: {imputed_count:>3} values ({success_rate:>5.1f}% success)")
        print()

# Overall summary
total_original = sum([s['original_missing'] for s in features_summary])
total_final = sum([s['final_missing'] for s in features_summary])
total_imputed = sum([s['imputed_count'] for s in features_summary])

print(f"  OVERALL IMPUTATION PERFORMANCE:")
print(f"   Original missing values: {total_original:,}")
print(f"   Successfully imputed: {total_imputed:,}")
print(f"   Remaining missing: {total_final:,}")
print(f"   Overall success rate: {(total_imputed/total_original*100):.1f}%")

# Data quality indicators
price_range = listings_imputed[listings_imputed['current_price'] > 0]['current_price']
surface_range = listings_imputed[listings_imputed['surface_m2'] > 0]['surface_m2']

print(f"\nDATA QUALITY AFTER IMPUTATION:")
print(f"   Price range: €{price_range.min():,.0f} - €{price_range.max():,.0f}")
print(f"   Price median: €{price_range.median():,.0f}")
print(f"   Surface range: {surface_range.min():.0f}m² - {surface_range.max():.0f}m²")
print(f"   Surface median: {surface_range.median():.0f}m²")
print(f"   Room distribution: {dict(listings_imputed['room_count'].value_counts().sort_index())}")

current_price:
   Before:    5 missing (  0.4%)
   After:     0 missing (  0.0%)
   Imputed:   5 values (100.0% success)

surface_m2:
   Before:    0 missing (  0.0%)
   After:     0 missing (  0.0%)
   Imputed:   0 values (100.0% success)

room_count:
   Before:    3 missing (  0.2%)
   After:     0 missing (  0.0%)
   Imputed:   3 values (100.0% success)

floor_count:
   Before:  474 missing ( 33.2%)
   After:     0 missing (  0.0%)
   Imputed: 474 values (100.0% success)

  OVERALL IMPUTATION PERFORMANCE:
   Original missing values: 482
   Successfully imputed: 482
   Remaining missing: 0
   Overall success rate: 100.0%

DATA QUALITY AFTER IMPUTATION:
   Price range: €12,000 - €485,000
   Price median: €188,000
   Surface range: 11m² - 179m²
   Surface median: 63m²
   Room distribution: {1.0: 28, 2.0: 263, 3.0: 547, 4.0: 487, 5.0: 81, 6.0: 16, 7.0: 6}


In [10]:
# Create processed directory if it doesn't exist
processed_dir = Path("../data/processed")
processed_dir.mkdir(exist_ok=True)

# Save imputed dataset
output_file = processed_dir / "listings_imputed.csv"
listings_imputed.to_csv(output_file, index=False)

print(f"Saved imputed dataset: {output_file}")
print(f"Dataset info:")
print(f"Rows: {len(listings_imputed):,}")
print(f"Columns: {len(listings_imputed.columns)}")
print(f"Size: {output_file.stat().st_size / 1024 / 1024:.1f} MB")

# Save imputation summary
summary_file = processed_dir / "imputation_summary.csv"
pd.DataFrame(features_summary).to_csv(summary_file, index=False)
print(f"Saved imputation summary: {summary_file}")

# Display final columns
print(f"\nFINAL DATASET COLUMNS:")
for i, col in enumerate(listings_imputed.columns):
    print(f"   {i+1:2d}. {col}")

Saved imputed dataset: ../data/processed/listings_imputed.csv
Dataset info:
Rows: 1,428
Columns: 15
Size: 0.7 MB
Saved imputation summary: ../data/processed/imputation_summary.csv

FINAL DATASET COLUMNS:
    1. listing_id
    2. surface_m2
    3. description
    4. room_count
    5. floor
    6. floor_count
    7. current_price
    8. city
    9. city_zip
   10. transaction_type
   11. item_type
   12. price_was_imputed
   13. surface_was_imputed
   14. rooms_was_imputed
   15. floor_count_was_imputed
